<div style="display: flex; align-items: center;">
  <img src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/DecisionIntelligenceSemanticKernelLogo.png" width="60px" style="margin-right: 10px;">
  <span style="font-size: 1.5em; font-weight: bold;">Semantic Kernel - Chat Completion for Decisions</span>
</div>

Decision Intelligence applied in this module:  
* Listing of various decision-making frameworks and with their descriptions by using Chat Completion  
* Decision Scenario: Recommendation of a decision-making framework for specific higher-stakes decisions (purchase of a car)  
* Illustrating a domain specific decision-making framework (millitary intelligence)  


In this module, the Semantic Kernel ability to create a chat experience will be introduced. This is a much richer experience than just sending simple prompts that are stateless and context is forgotten.

Semantic Kernel has first-class support for chat scenarios, where the user talks back and forth with the LLM, the arguments get populated with the history of the conversation. During each new run of the kernel, the arguments will be provided to the AI with content. This allows the LLM to know the historical context of the conversation.

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
// Import the required NuGet configuration packages
#r "nuget: Microsoft.Extensions.Configuration, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"

using Microsoft.Extensions.Configuration.Json;
using Microsoft.Extensions.Configuration;
using System.IO;

// Load the configuration settings from the local.settings.json and secrets.settings.json files
// The secrets.settings.json file is used to store sensitive information such as API keys
var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration, 9.0.6 Microsoft.Extensions.Configuration.Json, 9.0.6

In [5]:
// Install the required NuGet packages
// Note: This also installs the Dependency Injection Package to retrieve the ChatCompletionService
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.6"
#r "nuget: Microsoft.SemanticKernel, 1.59.0"

using Microsoft.Extensions.DependencyInjection.Extensions;
using Microsoft.Extensions.DependencyInjection;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

var chatCompletionService = semanticKernel.Services.GetRequiredService<IChatCompletionService>();

Installed Packages Microsoft.Extensions.DependencyInjection, 9.0.6 Microsoft.SemanticKernel, 1.59.0

Using Azure OpenAI Service


### Step 2 - Execute Decision Chat Completion with Prompt Execution Settings 

> 📜 **_"If you can’t describe what you are doing as a process, you don’t know what you’re doing."_** 
>
> -- <cite>W. Edwards Deming (renowned American engineer, statistician, and management consultant)</cite>  


Using the Semantic Kernel ChatCompletion service is very similar to inovoking a prompt for basic LLM interactions. The chat completion service will provide very similar results to invoking the prompt directly.

In [6]:
// Simple prompt to list some decision frameworks this GenAI LLM is familiar with 
// LLMs are trained on a diverse range of data and can provide insights on a wide range of topics like decision frameworks
// SLMS (smaller LLMs) are trained on a more specific range of data and may not provide insights on all topics
var simpleDecisionPrompt = """
Provide a list of 5 decision frameworks that can help improve the quality of decisions.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Create a new OpenAI prompt execution settings object
var openAIPromptExecutionSettings = new OpenAIPromptExecutionSettings { 
    MaxTokens = 1000, 
    Temperature = 0.1, 
    TopP = 1.0, 
    FrequencyPenalty = 0.0, 
    PresencePenalty = 0.0
    };

// Execute the chat completion request using the prompt and settings defined above
var chatResult = await chatCompletionService.GetChatMessageContentAsync(simpleDecisionPrompt, openAIPromptExecutionSettings);

// Display the result of the chat completion request as Markdown
chatResult.Content.DisplayAs("text/markdown");

### 1. SWOT Analysis  
A structured framework for evaluating the Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

### 2. Decision Matrix (Weighted Scoring Model)  
A tool that helps compare multiple options by scoring them against defined criteria, often with assigned weights to reflect importance.

### 3. Cost-Benefit Analysis  
A systematic approach to estimating the strengths and weaknesses of alternatives by comparing the expected costs and benefits.

### 4. The OODA Loop  
A cyclical process consisting of Observe, Orient, Decide, and Act, designed to enable rapid and adaptive decision-making.

### 5. Six Thinking Hats  
A method that encourages looking at a decision from multiple perspectives (logic, emotion, caution, optimism, creativity, and process) to ensure a well-rounded evaluation.

### Step 3 - Execute Decision Chat Completion with Streaming 

Semantic Kernel supports prompt response streaming in the ChatCompletionService. This allows responses to be streamed to the client as soon as they are made available by the LLM and API services. Below the same decision prompt is executed. However, notice that chunks are streamed and can be read by the user as soon as they appear. 

In [7]:
// Stream the chat completion response
await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(simpleDecisionPrompt, openAIPromptExecutionSettings))
{
    Console.Write(content);
}

### 1. SWOT Analysis  
A structured planning method that evaluates Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.

### 2. Decision Matrix (Weighted Scoring Model)  
A tool that helps compare multiple options by scoring them against a set of criteria, often with assigned weights to reflect importance.

### 3. Cost-Benefit Analysis  
A systematic approach to estimating the strengths and weaknesses of alternatives by comparing the expected costs and benefits.

### 4. The OODA Loop  
A cyclical process of Observe, Orient, Decide, and Act, designed to enable rapid and adaptive decision-making.

### 5. Six Thinking Hats  
A framework that encourages looking at a decision from multiple perspectives (logic, emotion, caution, optimism, creativity, and process) to ensure a well-rounded evaluation.

### Step 4 - Execute Decision Chat Completion with a System Prompt & Chat History 

Semantic Kernel has a ChatHistory object that can be used with the ChatCompletionService to provide historical chat context to the LLM. Notice that the ChatHistory object differentiates between the different types of chat messages:
* System Message - System or MetaPrompt. These are usually global instructions that set the "overall rules" for interacting with the LLMs.
* User Message - A message from the user
* Assistant Message - A message from the LLM. This is a message generated from an assistant or an agent. 

Identifying the messages from which role (user) it came from can help the LLM improve its own reasoning responses. This is a more sophisticated approach than passing chat history in a long dynamic string. ChatHistory objects can be serialized and persisted into databases as well.

In [8]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPrompt = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

Output Format Instructions:
When generating Markdown, do not use any headings higher than ###. 
Avoid # and ## headers. Use only ###, ####, or lower-level headings if necessary. 
All top-level section headers should start at ### or lower. 
""";

// Simple instruction prompt to list 5 (five) decision frameworks this GenAI LLM is familiar with
var simpleDecisionPrompt = """
Provide five Decision Frameworks that can help improve the quality of decisions.
""";

// Create a new chat history object with proper system and user message roles
var chatHistory = new ChatHistory();
chatHistory.AddSystemMessage(systemPrompt);
chatHistory.AddUserMessage(simpleDecisionPrompt);

// Note: The chat completion service can also be used with a chat history object, which allows for a more conversational context.
var chatResponse = string.Empty;

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add the assistant's response to the chat history
chatHistory.AddAssistantMessage(chatResponse);

### 1. SWOT Analysis  
**Purpose**: To evaluate the Strengths, Weaknesses, Opportunities, and Threats related to a decision or project.  
**How it Helps**: Offers a structured way to identify internal and external factors that can impact your decision quality, and guides resource allocation and risk mitigation.

*Example*:  
- Strengths: What advantages does the decision offer?  
- Weaknesses: Where are the vulnerabilities?  
- Opportunities: What external options could improve outcomes?  
- Threats: What risks threaten success?

---

### 2. Cost-Benefit Analysis  
**Purpose**: To compare the costs and benefits of different options quantitatively or qualitatively.  
**How it Helps**: Promotes objective evaluation, ensuring that benefits outweigh costs and helping prioritize among alternatives.

*Example*:
- List estimated costs (monetary, time, resources).
- Enumerate expected benefits (profit, efficiency, satisfaction).
- Calculate net gain/loss for each option.

---

### 3. Decision Matrix (Weighted Scoring Model)  
**Purpose**: To evaluate multiple alternatives against defined criteria, assigning weights to reflect importance.  
**How it Helps**: Improves objectivity and transparency, enabling complex decisions to be broken into manageable comparisons.

*Example*:
- Criteria: Cost (40%), Quality (30%), Speed (30%)
- Score each alternative on each criterion.
- Multiply by weights, sum scores, and choose the highest.

---

### 4. Six Thinking Hats  
**Purpose**: To systematically explore different perspectives around a decision using six metaphorical “hats” (modes of thinking).  
**How it Helps**: Avoids groupthink, surfaces diverse views, and ensures comprehensive analysis.

*Example*:
- White Hat: Facts & data
- Red Hat: Emotions & intuition
- Black Hat: Risks & cautions
- Yellow Hat: Benefits & optimism
- Green Hat: Creativity & alternatives
- Blue Hat: Process & organization

---

### 5. OODA Loop (Observe, Orient, Decide, Act)  
**Purpose**: Originally developed for military decisions, this iterative cycle suits fast-changing environments.  
**How it Helps**: Encourages rapid feedback, continuous learning, and adaptation as new information arises.

*Steps*:
1. **Observe** the situation and context.
2. **Orient** by analyzing information and contextual factors.
3. **Decide** on the best course of action.
4. **Act** on the decision, then loop back to observe results.

---

**Choosing the Right Framework:**  
Select based on your decision’s complexity, urgency, and the information available. Combining frameworks (e.g., SWOT with a Decision Matrix) can further strengthen decision quality.

In the next step, an additional prompt instruction will be added to the chat history. From the 5 decision frameworks provided in the chat history, Generative AI is asked to recommed a framework best suited for the militaty intelligence community. Notice that the previous chat history is automatically provided to provide additional intelligence context. 

> 📜 **_"Take time to deliberate, but when the time for action comes, stop thinking and go in."_**
>
> -- <cite>Napoleon Bornaparte (French Emporer and brilliant military mind)</cite>  

In [9]:
// Note: No reference is made to what previous frameworks were listed.
// Note: Previous context is maintained by the Semantic Kernel ChatHistory object 
var simpleDecisionPromptFollowupQuestionPartTwo = """
Which of the 5 decision frameworks listed above is best suited for the military intelligence community?  
Think carefully step by step about what decision frameworks are needed to answer the query.  
Select only the single best framework. 
""";

// Add User message to the chat history
chatHistory.AddUserMessage(simpleDecisionPromptFollowupQuestionPartTwo);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistory))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistory.AddAssistantMessage(chatResponse);

### Step-by-Step Reasoning

#### 1. **Understand the Military Intelligence Decision Context**
- **Characteristics**:
  - High-stakes, time-sensitive situations
  - Incomplete and evolving information
  - Need for rapid response and adaptability
  - Potential for adversarial counteractions
  - Importance of continuous feedback and learning
- **Core Needs**: Agility, adaptability, iterative learning, and speed in decision cycles

#### 2. **Assess Each Framework’s Suitability**

- **SWOT Analysis**: 
  - Useful for strategic overviews, but less effective for fast-changing, tactical environments.
  - Not inherently iterative or rapid.

- **Cost-Benefit Analysis**: 
  - Focuses on optimizing outcomes using available data.
  - Slower, relies on relatively complete information, more applicable for planning than real-time operations.

- **Decision Matrix**: 
  - Structured comparisons, suitable for complex choices where criteria are clear and stable.
  - Less agile, not designed for dynamic, rapidly shifting contexts.

- **Six Thinking Hats**: 
  - Fosters comprehensive group analysis.
  - Valuable for brainstorming or planning, but not optimized for the speed and iteration needed in operations.

- **OODA Loop**: 
  - Designed for high-pressure, dynamic environments.
  - Allows for rapid observation, orientation, agile decision-making, and immediate implementation.
  - Iterative; accommodates new intelligence, battles evolving threats, and adjusts in real time.

#### 3. **Evaluate Against Military Intelligence Needs**
- The military intelligence community often deals with real-time threats, incomplete datasets, and adversarial activity. The ability to observe, analyze, decide, and act quickly — and repeatedly — is crucial.
- An iterative and agile cycle is far more important than a one-off, comprehensive analysis or static evaluation.

#### 4. **Conclusion**

The **OODA Loop** (Observe, Orient, Decide, Act) is the single best decision framework for the military intelligence community. It aligns directly with the community’s operational demands for speed, adaptability, and continuous feedback in a constantly changing environment. 

**Summary**:  
**OODA Loop** best supports the military intelligence community’s need for rapid, iterative, and adaptive decision-making in high-pressure, uncertain, and competitive contexts.

### Step 5 - Execute Decision Chat Completion with a Multi-Turn Conversation Scenario 

> 📜 **_"Price is what you pay. Value is what you get."_**  
>
> -- <cite>Warren Buffet (one of the most successful investors of all time)</cite>  

Semantic Kernel can maintain state with the ChatHistory object. This allows natural converation between the user and AI to be persisted for future use. It can be saved for future reference, it can be replayed, it can be forked in certain spots to try different scenarios, it can be used as a starting point for other conversations etc. 

In the scenario below, let's build a multi-turn conversation about a decision over a purchase for a fitness tracker. Notice that each turn the appropriate Chat History roles are populated with the appropriate prompts and assistant (AI) responses. This in effect acts as the Gathering Intelligence step of the Decision Intelligence framework as the conversation information has been persisted and can be re-used as context for the decision scenario.

In [10]:
// Set the overall system prompt to behave like a decision intelligence assistant (persona)
var systemPromptDecisionConversation = """
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.
""";

// User prompt to start a decision conversation about buying a fitness tracker
var userPromptDecisionConversationTurn1 = """
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 
""";

var assistantResponseDecisionConversationTurn1 = """
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.
""";

var userPromptDecisionConversationTurn2 = """
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?
""";

var assistantResponseDecisionConversationTurn2 = """
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.
""";

var userPromptDecisionConversationTurn3 = """
Are there any popular models you would recommend for that balance of battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn3 = """
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.
""";

var userPromptDecisionConversationTurn4 = """
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?
""";

var assistantResponseDecisionConversationTurn4 = """
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 
""";

// Create a new chat history with a multi-turn conversation about buying a fitness tracker
// Notice how the conversation flow uses different roles (user and assistant) to simulate a realistic decision-making dialogue
var chatHistoryDecisionConversation = new ChatHistory();
chatHistoryDecisionConversation.AddSystemMessage(systemPromptDecisionConversation);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn1);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn1);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn2);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn2);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn3);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn3);
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationTurn4);
chatHistoryDecisionConversation.AddAssistantMessage(assistantResponseDecisionConversationTurn4);


Using the above conversation history as the "Gathered Intelligence" betweeen the user and the AI assistant, let's make a final decision and ask for final feedback and a decision evaluation. This "Gathered Intelligence" for the decision can be persisted, re-loaded, simulated multiple times, applied to different AI systems to optimize decisions further.  

In [11]:
var userPromptDecisionConversationFinalDecision = """
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptDecisionConversationFinalDecision);

// Render Markdown or stream the chat completion response
var renderMarkdown = true;
if (renderMarkdown)
{
    // Get the chat completion response as a single content object
    var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
    chatResponse = chatContent.Content;
    chatResponse.DisplayAs("text/markdown");
}
else
{
    // Stream the chat completion response
    await foreach (var content in chatCompletionService.GetStreamingChatMessageContentsAsync(chatHistoryDecisionConversation))
    {
        chatResponse += content;
        Console.Write(content);
    }
}

// Add Assistant message to the chat history
chatHistoryDecisionConversation.AddAssistantMessage(chatResponse);

Great choice! The **Fitbit Inspire 3** is a widely praised model for those seeking long battery life, solid sleep tracking, and affordability. Here are a few final thoughts and tips to help you get the most out of your decision:

### Positives
- **Battery Life:** Up to 10 days between charges is realistic, enabling consistent sleep tracking without nightly recharging.
- **Sleep Tracking:** Offers detailed breakdowns of your sleep stages, plus an overall sleep score and insights through the Fitbit app.
- **Comfort and Design:** Lightweight and slim, comfortable for day and night wear.

### Tips for Maximizing Your Experience
- **App Ecosystem:** Be sure to explore the Fitbit app’s features—its dashboard offers intuitive data visualizations and actionable sleep tips.
- **Firmware Updates:** Check for firmware updates after purchase; these can enhance features and fix bugs.
- **Replacement Bands:** Consider a second, softer band for overnight use to maximize comfort.
- **Privacy:** Set privacy and data-sharing preferences when setting up the app.

### Limitations to Note
- **GPS:** The Inspire 3 does not have built-in GPS but will use your phone’s GPS for outdoor activities if desired.
- **Advanced Metrics:** While robust for most, if you want even more detailed wellness metrics down the road, you might look at higher-end models or Fitbit’s Premium subscription.

**Summary:**  
For your emphasis on battery life, robust sleep tracking, and value for money, the Inspire 3 is an excellent and practical choice. Enjoy putting your new tracker to work for your health goals! If you have further questions or want advice on setup/usage, feel free to ask.

### Step 6 - Inspect & Optimize Gathered Intelligence of Chat History 

The Semantic Kernel Chat History is a transparent construct that can be inspected and written out. In fact, the ChatHistory object can be manipulated to replay chats from middle interactions to simulate different outcomes.

Execute the cell below to write out the Semantic Kernel Chat History of this chat.

> **📝 Note:** In the Decision Intelligence framework, chat history can serve as a form of gathered intelligence. Interactions among users, AI models, processes, and agents provide valuable context for effective decision-making. It’s highly recommended to persist chat history objects (agent threads), especially during decision optimization, to ensure critical information is retained and accessible.  

In [12]:
// Print the number of chat interactions and the chat history (turns)
Console.WriteLine("Number of chat interactions: " + chatHistoryDecisionConversation.Count());

// Change this to a string builder and show as markdown
var stringBuilderChatHistory = new StringBuilder();
foreach (var message in chatHistoryDecisionConversation)
{
    // add a new line for each message
    stringBuilderChatHistory.AppendLine($"**{message.Role.ToString().ToUpper()}**:");
    stringBuilderChatHistory.Append($"{message.Content.Replace("#", string.Empty)}");
    stringBuilderChatHistory.AppendLine("\n");
}

// Display the chat history as Markdown
stringBuilderChatHistory.ToString().DisplayAs("text/markdown");

Number of chat interactions: 11


**SYSTEM**:
You are a Decision Intelligence assistant. 
Assist the user in exploring options, reasoning through decisions, problem-solving, and applying systems thinking to various scenarios. 
Provide structured, logical, and comprehensive advice.

**USER**:
I am considering buying a fitness tracker but I am not sure which features really matter. 
Could you explain the main considerations? 

**ASSISTANT**:
Absolutely! Key considerations for choosing a fitness tracker often include: 
- Fitness Features (e.g., step counting, heart-rate monitoring, workout tracking)
- Health Monitoring (like sleep tracking or advanced ECG capabilities)
- Battery Life
- Comfort and Fit
- Compatibility with your smartphone's operating system
- Price Range
- Knowing which of these you value most will help narrow down your options.

**USER**:
Got it. Battery life and sleep tracking are both important to me. 
What's a typical battery life range for trackers, and how good is their sleep tracking?

**ASSISTANT**:
Different fitness trackers have varying battery lives, typically anywhere from 2-3 days (for more feature-rich or bright-display devices) 
up to about 10-14 days (for more minimalist trackers). 

Regarding sleep tracking:
- Many modern devices can track different sleep stages (light, deep, REM).
- High-end devices even track breathing rate and oxygen saturation during the night.
- Some also offer insights about sleep quality and tips for improvement.

If battery life is a priority, you may want to consider trackers with a more conservative display and fewer power-hungry features.

**USER**:
Are there any popular models you would recommend for that balance of battery life and sleep tracking?

**ASSISTANT**:
Popular options that strike a balance between robust sleep tracking and good battery life include:
- Fitbit Inspire 3 or Fitbit Charge 5: Both have solid sleep tracking, multi-day battery, and strong app support.
- Garmin Vivosmart 5: Offers decent fitness and sleep insights, plus better-than-average battery life.
- Xiaomi Mi Band 7: Budget-friendly, surprisingly robust health metrics, and a long-lasting battery.

Each varies in price and features, so consider your budget and the specific metrics you’d like to track.

**USER**:
That is great infor. 
I also have a friend who said I should consider an Apple Watch. 
Does that have comparable battery life and sleep tracking?

**ASSISTANT**:
The Apple Watch is known for its extensive feature set, including advanced health monitoring, 
but its battery life tends to be shorter—usually around 18-24 hours on a single charge 
(or possibly a bit more in low-power mode). While its sleep tracking features have improved over time, 
you might find you need to charge it daily or almost daily, which could conflict with your goal of 24/7 tracking. 

**USER**:
Thank you for all of that information, I think I'll go with a Fitbit. 
The Fitbit Inspire 3 seems like a good fit for my budget and needs. 

Any quick final thoughts or insights on my decision? 

**ASSISTANT**:
Great choice! The **Fitbit Inspire 3** is a widely praised model for those seeking long battery life, solid sleep tracking, and affordability. Here are a few final thoughts and tips to help you get the most out of your decision:

 Positives
- **Battery Life:** Up to 10 days between charges is realistic, enabling consistent sleep tracking without nightly recharging.
- **Sleep Tracking:** Offers detailed breakdowns of your sleep stages, plus an overall sleep score and insights through the Fitbit app.
- **Comfort and Design:** Lightweight and slim, comfortable for day and night wear.

 Tips for Maximizing Your Experience
- **App Ecosystem:** Be sure to explore the Fitbit app’s features—its dashboard offers intuitive data visualizations and actionable sleep tips.
- **Firmware Updates:** Check for firmware updates after purchase; these can enhance features and fix bugs.
- **Replacement Bands:** Consider a second, softer band for overnight use to maximize comfort.
- **Privacy:** Set privacy and data-sharing preferences when setting up the app.

 Limitations to Note
- **GPS:** The Inspire 3 does not have built-in GPS but will use your phone’s GPS for outdoor activities if desired.
- **Advanced Metrics:** While robust for most, if you want even more detailed wellness metrics down the road, you might look at higher-end models or Fitbit’s Premium subscription.

**Summary:**  
For your emphasis on battery life, robust sleep tracking, and value for money, the Inspire 3 is an excellent and practical choice. Enjoy putting your new tracker to work for your health goals! If you have further questions or want advice on setup/usage, feel free to ask.



By persisting the Decision Intelligence chats and making them available for inspection, can further help the decision-making process. For example you can load the Chat History summarize it or even have AI recommend optimizations for it based on the conversation. 

> **📝 Note:** The process below is simplified to illustrate the power of Gathering Intelligence with user <--> AI interactions. More advanced best practices will be introduced in further sections. 

In [13]:
var userPromptSummarizeTheDecisionChat = """
Briefly summarize the chat conversation about buying a fitness tracker.
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptSummarizeTheDecisionChat);

var chatContent = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
chatResponse = chatContent.Content;
chatResponse.DisplayAs("text/markdown");

// Optimize the chat history for any future decision interactions
var userPromptOptimizeTheDecisionChat = """
Based on the chat conversation so far...
Provide some recommendations on how the decision interactions could be optimized for decision-making.
What are some considerations for the AI to help the use make even better decisions?
""";

// Add User message to the chat history
chatHistoryDecisionConversation.AddUserMessage(userPromptOptimizeTheDecisionChat);

var chatContentOptimization = await chatCompletionService.GetChatMessageContentAsync(chatHistoryDecisionConversation);
var chatResponseOptimization = chatContentOptimization.Content;
chatResponseOptimization.DisplayAs("text/markdown");

You asked for guidance on choosing a fitness tracker, highlighting battery life and sleep tracking as key priorities. We discussed battery life ranges (2–14 days), how well current trackers monitor sleep, and identified several models (Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, Xiaomi Mi Band 7) that balance these needs. You also asked about the Apple Watch, learning that its battery life is shorter and may require daily charging. Based on this information, you decided on the Fitbit Inspire 3, and I confirmed it is a good fit for your needs with a few tips for optimal use.

**Summary of the Chat Conversation:**
You were considering buying a fitness tracker and wanted to understand which features matter most. You highlighted battery life and sleep tracking as important to you. After learning about typical battery life ranges and sleep tracking capabilities, you asked for model recommendations. You were advised on models like the Fitbit Inspire 3, Fitbit Charge 5, Garmin Vivosmart 5, and Xiaomi Mi Band 7. You also inquired about the Apple Watch and learned its battery life may be insufficient for your needs. Ultimately, you chose the Fitbit Inspire 3, and requested final insights before purchase.

---

**Recommendations to Optimize Decision Interactions:**

1. **Structured Decision Frameworks:**
   - The AI could use a decision matrix or comparison tables to objectively evaluate different models against your prioritized features, helping clarify trade-offs.

2. **Clarifying Priorities and Constraints:**
   - The AI could have asked more questions up front to further clarify your must-haves, nice-to-haves, budget, and any style or ecosystem preferences.

3. **Personalization:**
   - The AI could request information about your specific activity habits, health goals, or preferred brands to further tailor suggestions.

4. **Scenario-Based Reasoning:**
   - The AI could offer usage scenarios (e.g., how often you exercise, whether you prefer charging overnight, etc.) to help you visualize how features align with your lifestyle.

5. **Evidence and User Feedback:**
   - Bringing in recent user reviews, satisfaction ratings, or reliability data could add practical context to technical recommendations.

6. **Long-Term Use Considerations:**
   - Discuss ongoing support (app updates, band replacements), subscription requirements, or resale value—factors that may be significant but are often overlooked.

7. **Decision Support Tools:**
   - Providing links to reputable comparison websites, or interactive tools/calculators, could empower you to continue your research confidently.

---

**Considerations for AI to Help Users Make Better Decisions:**

- **Active Listening:**  
  AI should summarize and confirm user requirements to ensure understanding before suggesting options.

- **Transparent Reasoning:**  
  Clearly explain why particular options are recommended, including trade-offs, rather than just listing features.

- **Iterative Exploration:**  
  Encourage users to refine or revisit their preferences as they learn more.

- **Bias Avoidance:**  
  Remain impartial; avoid over-recommending any one brand or model unless it strongly fits stated needs.

- **Future-Proofing:**  
  Help users anticipate how needs might evolve (e.g., changing fitness goals or increasing interest in advanced health metrics).

By implementing these strategies, AI can support users in making confident, well-informed, and personally relevant decisions.